In [ ]:
import numpy as np
import random
from tqdm import tqdm
from typing import List
import multiprocessing
from functools import partial
from util.data_generate import *
from util.methods import *

# ---------------------------- 初始化参数 ----------------------------
p = 50  # 科目总数
m = 8   # 需要选择的科目数
group_sizes = [10] * 5
seed, J, n_iter, num_repeats = 1, 10000, 150, 30
np.random.seed(seed)
Sigma1 = np.loadtxt("Sigma1.txt")
mu1 = 70 * np.ones(p)
mu2, std2, R2, Sigma2 = data2(p=p, seed=seed, group_sizes=group_sizes)
mu = np.random.multivariate_normal(mu1, Sigma1)

# ---------------------------- 模拟环境 ----------------------------
class ExamEnvironment:
    def __init__(self, n_subjects=50):
        np.random.seed(42)
        self.true_means = mu
        self.cov_matrix = np.loadtxt("Sigma1.txt")
        
    def observe_scores(self, selected_indices: List[int]) -> List[float]:
        ids = selected_indices
        return np.random.multivariate_normal(mu[ids], Sigma2[np.ix_(ids, ids)])

# ---------------------------- 动态概率编码器 ----------------------------
class DynamicProbabilityEncoder:
    def __init__(self, n_subjects: int):
        self.n_subjects = n_subjects
        self.prob_vector = np.ones(n_subjects) / n_subjects
    
    def update(self, sample_means: np.ndarray) -> None:
        """
        使用 Softmax 更新科目被选入候选集的概率：
        """
        # 1. 越小均值对应的值越大
        inverted_means = 80-sample_means
        
        # 2. 计算 Softmax（确保数值稳定）
        max_inverted = np.max(inverted_means)
        exp_values = np.exp((inverted_means - max_inverted))
        softmax_probs = exp_values / np.sum(exp_values)
        
        # 4. 归一化并更新概率
        self.prob_vector = softmax_probs / np.sum(softmax_probs)
        
    def sample_candidates(self, size: int) -> List[int]:
        return sorted(np.random.choice(
            self.n_subjects, size=size, replace=False, p=self.prob_vector
        ))

# ---------------------------- 模拟退火算法 ----------------------------
class MultiStartSA:
    def __init__(self, select_size=8, n_starts=6, inner_iters=100):
        self.select_size = select_size
        self.n_starts = n_starts      # 并行启动的SA数量
        self.inner_iters = inner_iters # 每个SA的内部迭代次数
        self.T_init = 100.0           # 初始温度
        self.cooling_rate = 0.95      # 冷却速率

    def fitness(self, solution: List[int], sample_means: np.ndarray) -> float:
        """计算最小值期望"""
        z_samples = np.random.randn(J, self.select_size)
        sub_Sigma = Sigma2[np.ix_(solution, solution)]
        U = np.linalg.cholesky(sub_Sigma)
        return np.mean(np.min(sample_means[solution] + (U @ z_samples.T).T, axis=1))

    def get_neighbor(self, current_solution: List[int], candidates: List[int]) -> List[int]:
        """邻域：随机替换一门科目"""
        neighbor = current_solution.copy()
        idx_to_replace = random.randint(0, self.select_size - 1)
        available = list(set(candidates) - set(neighbor))
        if available:
            neighbor[idx_to_replace] = random.choice(available)
        return sorted(neighbor)

    def run_single_sa(self, initial_solution: List[int], candidates: List[int], sample_means: np.ndarray) -> List[int]:
        """单个SA的独立运行"""
        current_solution = initial_solution.copy()
        current_energy = self.fitness(current_solution, sample_means)
        best_solution = current_solution.copy()
        best_energy = current_energy
        T = self.T_init

        for _ in range(self.inner_iters):
            neighbor = self.get_neighbor(current_solution, candidates)
            neighbor_energy = self.fitness(neighbor, sample_means)
            delta_E = neighbor_energy - current_energy

            # 接受条件
            if delta_E < 0 or random.random() < np.exp(-delta_E / T):
                current_solution = neighbor
                current_energy = neighbor_energy

            # 更新全局最优
            if current_energy < best_energy:
                best_solution = current_solution.copy()
                best_energy = current_energy

            T *= self.cooling_rate

        return best_solution

    def optimize(self, candidates: List[int], sample_means: np.ndarray) -> List[int]:
        """多起点SA主流程（串行版）"""
        # 生成多个初始解
        initial_solutions = [random.sample(candidates, self.select_size) 
                           for _ in range(self.n_starts)]

        # 运行多个SA
        results = []
        for sol in initial_solutions:
            best_sol = self.run_single_sa(sol, candidates, sample_means)
            results.append(best_sol)

        # 选择所有结果中适应度最好的解
        best_solution = min(results, key=lambda x: self.fitness(x, sample_means))
        return best_solution

# ---------------------------- 主优化流程 ----------------------------
csv_path = f'{p}_{m}_SA.csv'
csv_path1 = f'{p}_{m}_SA_pre.csv'
csv_path2 = f'{p}_{m}_SA_post.csv'

# 初始化CSV文件
with open(csv_path, 'w') as f:
    f.write(','.join(['Repetition'] + [f'Step_{i}' for i in range(1, n_iter + 1)]) + '\n')
with open(csv_path1, 'w') as f:
    f.write(','.join(['Repetition'] + [f'Step_{i}' for i in range(1, n_iter + 1)]) + '\n')
with open(csv_path2, 'w') as f:
    f.write(','.join(['Repetition'] + [f'Step_{i}' for i in range(1, n_iter + 1)]) + '\n')

# 初始化环境与组件
env = ExamEnvironment(n_subjects=50)
encoder = DynamicProbabilityEncoder(n_subjects=50)
ms_sa = MultiStartSA(select_size=8, n_starts=5, inner_iters=100)

for repeat in range(num_repeats):
    obs_num = np.zeros(p)
    history = {
        'best_groups': [],
        'prob_entropy': [],
        'sample_means': np.full(50, 70.0)  # 初始样本均值
    }
    G, G_pre, G_post = [], [], []

    for epoch in tqdm(range(n_iter)):
        # 1. 动态生成候选集
        candidates = encoder.sample_candidates(size=20)
        
        # 2. 运行模拟退火算法
        best_individual = ms_sa.optimize(candidates, history['sample_means'])
        
        # 3. 观测分数并更新
        observed_scores = env.observe_scores(best_individual)
        obs_num[best_individual] += 1
        for idx, score in zip(best_individual, observed_scores):
            history['sample_means'][idx] = 0.8 * history['sample_means'][idx] + 0.2 * score
        
        # 4. 更新概率编码器
        encoder.update(history['sample_means'])
        
        # 5. 记录
        history['best_groups'].append(best_individual)
        history['prob_entropy'].append(-np.sum(encoder.prob_vector * np.log(encoder.prob_vector + 1e-10)))

        z_samples = np.random.randn(J * 10, m)
        sub_Sigma = Sigma2[np.ix_(best_individual, best_individual)]
        U = np.linalg.cholesky(sub_Sigma)
        G.append(np.mean(np.min(mu[best_individual] + (U @ z_samples.T).T, axis=1)))

        Gmin = 1000
        idmin = []
        z_samples2 = np.random.randn(J, m)
        for id in history['best_groups']:
            sub_Sigma = Sigma2[np.ix_(id, id)]
            B = np.linalg.cholesky(sub_Sigma)
            Gtemp = np.mean(np.min(history['sample_means'][id] + (B @ z_samples2.T).T, axis=1))
            if Gtemp < Gmin:
                idmin = id.copy()
                Gmin = Gtemp
        ids1 = idmin

        sub_Sigma1 = Sigma2[np.ix_(ids1, ids1)]
        U1 = np.linalg.cholesky(sub_Sigma1)
        G_pre.append(np.mean(np.min(mu[ids1] + (U1 @ z_samples.T).T, axis=1)))

        U2, ids2 = improvement(p, m, history['sample_means'], std2, R2, Sigma2, J=J)
        G_post.append(np.mean(np.min(mu[ids2] + (U2 @ z_samples.T).T, axis=1)))
    
    # 保存结果
    U3, ids3 = improvement(p, m, mu, std2, R2, Sigma2, J=J)
    z_samples = np.random.randn(J * 10, m)
    G_real = np.mean(np.min(mu[ids3] + (U3 @ z_samples.T).T, axis=1))

    with open(csv_path, 'a') as f:
        f.write(','.join([str(repeat)] + [f'{x:.6f}' for x in np.array(G) - G_real]) + '\n')
    with open(csv_path1, 'a') as f:
        f.write(','.join([str(repeat)] + [f'{x:.6f}' for x in np.array(G_pre) - G_real]) + '\n')
    with open(csv_path2, 'a') as f:
        f.write(','.join([str(repeat)] + [f'{x:.6f}' for x in np.array(G_post) - G_real]) + '\n')

In [ ]:
mu

In [ ]:
history['sample_means']